# Transaction Risk Classification Project

This notebook processes transaction and alert data to classify transactions into risk groups based on sender/receiver patterns and transaction amounts.

## 1. Import Libraries and Set Display Options

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

: 

## 2. Load and Preprocess Alerts Data
- Extract `days_old` and `time` from `age_of_alert`
- Remove rows with missing `days_old`
- Filter alerts older than 30 days and drop duplicates

In [ ]:
alerts = pd.read_csv('exported_exported_34.csv', sep=';')
alerts[['days_old', 'time']] = alerts['age_of_alert'].str.extract(r'(\d+)\s+days,\s+([\d:.]+)')
alerts = alerts.dropna(subset=['days_old'])
alerts['days_old'] = alerts['days_old'].astype(int)
alerts = alerts[alerts['days_old'] > 30]
alerts = alerts.drop_duplicates()

## 3. Load and Clean Transaction Data
- Drop empty columns and rows with missing `STATE`


In [ ]:
tran = pd.read_csv('exported_exported_35.csv', sep=',')
tran = tran.dropna(axis=1, how='all')
tran = tran[tran['STATE'].notna()]

## 4. Explode Alert Transaction IDs
- Split `external_transaction_ids` by comma and explode to one row per tx_id
- Convert `tx_id` to integer for merging

In [ ]:
alerts['tx_id'] = alerts['external_transaction_ids'].str.split(r',\s*')
alerts = alerts.explode('tx_id').reset_index(drop=True)
alerts['tx_id'] = alerts['tx_id'].astype(int)

## 5. Merge Alerts with Transactions and Process Datetime
- Merge on `tx_id` and `rule_code`
- Drop duplicates
- Extract day of the week from transaction datetime

In [ ]:
all_data = alerts.merge(
    tran,
    how='left',
    left_on=['tx_id', 'rule_code'],
    right_on=['tx_id', 'CHECKS']
)
all_data = all_data.drop_duplicates()
all_data['tx_date_time'] = pd.to_datetime(all_data['tx_date_time'])
all_data['day_of_week'] = all_data['tx_date_time'].dt.dayofweek

## 6. Clean Sender Names and Create Business IDs
- Standardize sender names to lowercase and stripped
- Assign unique numeric `business_id`

In [ ]:
all_data['cleaned_sender_name'] = all_data['customer_name'].str.strip().str.lower().str.replace(',', '', regex=False)
all_data['business_id'] = all_data['cleaned_sender_name'].astype('category').cat.codes + 1

## 7. Step 1: Dataset of Not Suspicious / Soft Stop Transactions
- Group by sender and counterparty
- Compute count, median amount, MAD, and mode day of week
- Only include counterparty with >2 transactions

In [ ]:
n_s = all_data[(all_data['ACTIONS'] == 'Soft Stop') & (all_data['STATE'] == 'Not Suspicious')]
step_1_summary_dict = {}

for sender in n_s['cleaned_sender_name'].unique():
    info = n_s[n_s['cleaned_sender_name'] == sender]
    step_1_summary = info.groupby('counterparty_name').agg(
        count=('tx_base_amount', 'count'),
        avg_amount=('tx_base_amount', 'median'),
        std_amount=('tx_base_amount', lambda x: np.median(np.abs(x - np.median(x)))),
        avg_day_of_week=('day_of_week', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    ).reset_index()
    step_1_summary = step_1_summary[step_1_summary['count'] > 2]
    step_1_summary_dict[sender] = step_1_summary

## 8. Step 2: Receiver-based Summary
- Similar to step 1 but grouped by counterparty only

In [ ]:
step_2_summary = n_s.groupby('counterparty_name').agg(
    count=('tx_base_amount', 'count'),
    avg_amount=('tx_base_amount', 'median'),
    std_amount=('tx_base_amount', lambda x: np.median(np.abs(x - np.median(x)))),
    avg_day_of_week=('day_of_week', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
).reset_index()
step_2_summary = step_2_summary[step_2_summary['count'] > 2]

## 9. Identify In-Review Soft Stop Transactions

In [ ]:
i_r = all_data[(all_data['status'] == 'In Review') & (all_data['ACTIONS'] == 'Soft Stop')]
group_1 = []
group_2 = []
group_3 = []

## 10. Define Risk Classification Function
- Step 1: Sender-based check
- Step 2: Receiver-based check
- Step 3: Assign to group 3 if all else fails

In [ ]:
def risk(row):
    sender = row['cleaned_sender_name']
    receiver = row['counterparty_name']
    amount = row['tx_base_amount']
    day = row['day_of_week']
    risk = 0

    if sender in step_1_summary_dict:
        sender_info = step_1_summary_dict[sender]
        if receiver in sender_info['counterparty_name'].values:
            receiver_row = sender_info[sender_info['counterparty_name'] == receiver]
            median_amount = receiver_row['avg_amount'].values[0]
            mad_amount = receiver_row['std_amount'].values[0]
            mode_day = receiver_row['avg_day_of_week'].values[0]
            amount_ok = amount < (median_amount + mad_amount)
            day_ok = day == mode_day
            if amount_ok and day_ok:
                group_1.append(row.copy())
                return
            else:
                risk += 1
        else:
            risk += 1
    else:
        risk += 1

    if risk > 0 and receiver in step_2_summary['counterparty_name'].values:
        receiver_row = step_2_summary[step_2_summary['counterparty_name'] == receiver]
        median_amount = receiver_row['avg_amount'].values[0]
        mad_amount = receiver_row['std_amount'].values[0]
        mode_day = receiver_row['avg_day_of_week'].values[0]
        amount_ok = amount < (median_amount + mad_amount)
        day_ok = day == mode_day
        if amount_ok and day_ok:
            group_2.append(row.copy())
            return
        else:
            risk += 1

    group_3.append(row.copy())

## 11. Apply Risk Classification

In [ ]:
i_r.apply(risk, axis=1)

## 12. Combine Risk Groups into Single DataFrame

In [ ]:
df1 = pd.DataFrame(group_1)
df2 = pd.DataFrame(group_2)
df3 = pd.DataFrame(group_3)

df1['risk_group'] = 1
df2['risk_group'] = 2
df3['risk_group'] = 3

all_tx = pd.concat([df1, df2, df3], ignore_index=True)

## 13. Classify Alerts Based on Risk Groups

In [ ]:
alert_group_1 = []
alert_group_2 = []
alert_group_3 = []

unique_alerts = all_tx['alert_id'].unique()

for alert_id in unique_alerts:
    alert_rows = all_tx[all_tx['alert_id'] == alert_id]
    risk_groups = alert_rows['risk_group'].tolist()
    if all(r == 1 for r in risk_groups):
        alert_group_1.append(alert_id)
    elif 3 in risk_groups:
        alert_group_3.append(alert_id)
    elif all(r in [1, 2] for r in risk_groups) and 2 in risk_groups:
        alert_group_2.append(alert_id)